In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os
import shutil
import unicodedata
#import mlbgame
import glob

In [2]:
df = pd.read_csv('GL2017.csv')

In [3]:
columns = ['Date', 'Day', 'AwayTeam', 'AwayTeamGameNumber', 'HomeTeam', 'HomeTeamGameNumber', 'AwayScore', 'HomeScore',
          'AwayAtBats', 'AwayHits', 'AwayDoubles', 'AwayTriples', 'AwayHomeRuns', 'AwayRBI', 'AwayWalks', 
          'AwayIntentionalWalks', 'AwayStrikeouts', 'AwayStolenBases', 'AwayCaughtStealing', 'AwayLeftOnBase',
          'AwayIndividualERA', 'AwayTeamERA', 'AwayErrors', 'AwayStartingPitcher',
          'HomeAtBats', 'HomeHits', 'HomeDoubles', 'HomeTriples', 'HomeHomeRuns', 'HomeRBI', 'HomeWalks', 
          'HomeIntentionalWalks', 'HomeStrikeouts', 'HomeStolenBases', 'HomeCaughtStealing', 'HomeLeftOnBase',
          'HomeIndividualERA', 'HomeTeamERA', 'HomeErrors', 'HomeStartingPitcher']

In [4]:
df = df[columns]
df.head()

,Date,Day,AwayTeam,AwayTeamGameNumber,HomeTeam,HomeTeamGameNumber,AwayScore,HomeScore,AwayAtBats,AwayHits,...,HomeWalks,HomeIntentionalWalks,HomeStrikeouts,HomeStolenBases,HomeCaughtStealing,HomeLeftOnBase,HomeIndividualERA,HomeTeamERA,HomeErrors,HomeStartingPitcher
0,20170402,Sun,SFN,1,ARI,1,5,6,34,11,...,0,0,12,0,0,6,5,5,0,Zack Greinke
1,20170402,Sun,CHN,1,SLN,1,3,4,34,8,...,8,1,8,2,0,14,3,3,0,Carlos Martinez
2,20170402,Sun,NYA,1,TBA,1,3,7,36,9,...,3,0,10,1,0,9,2,2,1,Chris Archer
3,20170403,Mon,PHI,1,CIN,1,4,3,35,10,...,2,0,6,0,0,7,4,4,1,Scott Feldman
4,20170403,Mon,SDN,1,LAN,1,3,14,33,5,...,5,2,5,0,0,6,2,2,1,Clayton Kershaw


In [5]:
#removing the first 20 games of the season for each team
training_set = df.loc[df.AwayTeamGameNumber >20]
training_set = training_set.loc[df.HomeTeamGameNumber >20]

In [6]:
print (training_set.shape)
training_set.head()

(2123, 40)


,Date,Day,AwayTeam,AwayTeamGameNumber,HomeTeam,HomeTeamGameNumber,AwayScore,HomeScore,AwayAtBats,AwayHits,...,HomeWalks,HomeIntentionalWalks,HomeStrikeouts,HomeStolenBases,HomeCaughtStealing,HomeLeftOnBase,HomeIndividualERA,HomeTeamERA,HomeErrors,HomeStartingPitcher
280,20170424,Mon,SDN,21,ARI,21,6,7,37,10,...,2,0,8,4,0,6,5,5,1,Zack Greinke
289,20170425,Tue,SDN,22,ARI,22,3,9,37,10,...,3,0,8,2,1,9,3,3,0,Patrick Corbin
291,20170425,Tue,CIN,21,MIL,22,1,9,35,8,...,7,1,6,3,1,8,1,1,1,Zach Davies
293,20170425,Tue,LAN,21,SFN,21,2,1,32,7,...,1,0,9,1,0,8,2,2,0,Ty Blach
301,20170426,Wed,SDN,23,ARI,23,8,5,37,12,...,5,0,13,3,1,10,8,8,1,Zack Godley


In [7]:
"""X is the dataset. 
Returns a CSV containing last numOfGames games of team teamName (doesn't whether teamName is home or away).
Doesn't include currentGameNumber
"""
def get_last_n_games(teamName, numOfGames, currentGameNumber, X):
    away_games = df.loc[df.AwayTeam==teamName]
    away_games = away_games.loc[df.AwayTeamGameNumber < currentGameNumber]
    away_games = away_games.loc[df.AwayTeamGameNumber >= (currentGameNumber - numOfGames)]
    
    home_games = df.loc[df.HomeTeam==teamName]
    home_games = home_games.loc[df.HomeTeamGameNumber < currentGameNumber]
    home_games = home_games.loc[df.HomeTeamGameNumber >= (currentGameNumber - numOfGames)]
    
    return pd.concat([home_games, away_games])

In [8]:
"""
Possible values of column:
BATTING: BA, Runs, Walks, Strikeouts, LOB, StolenBases, OPS
"""
def return_rolling_average_of_column(df, team, column):
    numer = 0.0
    denom = 0
    away_games = df.loc[df.AwayTeam==team]
    home_games = df.loc[df.HomeTeam==team]
    
    if (column=="BA"): #we want the average batting average for this team
        awayhits = np.sum(away_games['AwayHits'])
        awayatbats = np.sum(away_games['AwayAtBats'])
        homehits = np.sum(home_games['HomeHits'])
        homeatbats = np.sum(home_games['HomeAtBats'])
        return (homehits + awayhits) / (homeatbats + awayatbats)
    
    elif (column=="Runs"):
        awayRuns = np.sum(away_games['AwayScore'])
        homeRuns = np.sum(home_games['HomeScore'])
        return (awayRuns + homeRuns) / df.shape[0]
    
    elif (column=="Walks"):
        awayWalks = np.sum(away_games['AwayWalks'])
        homeWalks = np.sum(home_games['HomeWalks'])
        return (awayWalks + homeWalks) / df.shape[0]
    
    elif (column=="Strikeouts"):
        away = np.sum(away_games['AwayStrikeouts'])
        home = np.sum(home_games['HomeStrikeouts'])
        return (away + home) / df.shape[0]
    
    elif (column=="StolenBases"):
        away = np.sum(away_games['AwayStolenBases'])
        home = np.sum(home_games['HomeStolenBases'])
        return (away + home) / df.shape[0]
    
    elif (column=="LOB"):
        away = np.sum(away_games['AwayLeftOnBase'])
        home = np.sum(home_games['HomeLeftOnBase'])
        return (away + home) / df.shape[0]
    
    elif (column=="OPS"): ##TO COMPLETE!!!!!!!
        #first calculating slugging 
        #slugging= (number singles)+ 2*(number doubles) + 3*(number triples)+ 4*(homeruns) / At Bats 
        home_doubles = np.sum(home_games['HomeDoubles'])
        away_doubles = np.sum(away_games['AwayDoubles'])
        home_triples = np.sum(home_games['HomeTriples'])
        away_triples = np.sum(away_games['AwayTriples'])
        home_hrs = np.sum(home_games['HomeHomeRuns'])
        away_hrs = np.sum(away_games['AwayHomeRuns'])
        homeatbats = np.sum(home_games['HomeAtBats'])
        awayatbats = np.sum(away_games['AwayAtBats'])
        home_hits = np.sum(home_games['HomeHits'])
        away_hits = np.sum(away_games['AwayHits'])
        home_walks = np.sum(home_games['HomeWalks'])
        away_walks = np.sum(away_games['AwayWalks'])
        home_singles = home_hits - (home_doubles + home_triples + home_hrs)
        away_singles = away_hits - (away_doubles  +away_triples + away_hrs)
        total_bases = (home_singles + away_singles) + (2* (home_doubles+away_doubles)) + (3* (home_triples+away_triples))+(4*(home_hrs+away_hrs))
        
        slugging = total_bases / (homeatbats + awayatbats)
        obp = (away_hits + home_hits + home_walks + away_walks) / (homeatbats + awayatbats + home_walks + away_walks)
        return (slugging + obp)
    
    #TODO: Calculate OBP and SLG and combine to get OPS

In [9]:
def return_cumulative_average_of_pitching_column(df, team, column, pitcher):
    # need walks, hits, strikeouts, ERA (all per game)
    away_games = df.loc[(df.AwayTeam==team) & (df.AwayStartingPitcher == pitcher)]
    home_games = df.loc[(df.HomeTeam==team) & (df.HomeStartingPitcher == pitcher)]  
    total_games = away_games.shape[0] + home_games.shape[0]
    #if pitcher hasn't played any games, just get average pitching stats for this team
    if total_games == 0:
        away_games = df.loc[(df.AwayTeam==team)]
        home_games = df.loc[(df.HomeTeam==team)] 
        total_games = away_games.shape[0] + home_games.shape[0]
    if(column == 'ERA'):
        away_ERA = np.sum(away_games['AwayTeamERA'])
        home_ERA = np.sum(home_games['HomeTeamERA'])
        return (away_ERA + home_ERA) / total_games
    elif(column == 'Walks'):
        away_walks = np.sum(away_games['HomeWalks'])
        home_walks = np.sum(home_games['AwayWalks'])
        return (away_walks + home_walks) / total_games
    elif(column == 'Hits'):
        away_hits = np.sum(away_games['HomeHits'])
        home_hits = np.sum(home_games['AwayHits'])
        return (away_hits + home_hits) / total_games
    elif(column == 'Strikeouts'):
        away_strikeouts = np.sum(away_games['HomeStrikeouts'])
        home_strikeouts = np.sum(home_games['AwayStrikeouts'])
        return (away_strikeouts + home_strikeouts) / total_games

In [10]:
h = get_last_n_games('TEX', 39, 40, training_set)
x = return_cumulative_average_of_pitching_column(h, 'TEX', 'ERA', 'Yu Darvish')
x

3.125

In [11]:
betting_data = pd.read_csv('Betting Data/2017.csv')
#make date same format
betting_data['date'] = betting_data.date.str.replace('/', '')
betting_data['date'] = '2017' + betting_data.date.str[:-2]
#only keep last names
betting_data['homePitcher'] = betting_data['homePitcher'].str.split().str[-1]
betting_data['awayPitcher'] = betting_data['awayPitcher'].str.split().str[-1]

current_game = training_set.iloc[0]
home_pitcher = current_game['HomeStartingPitcher']
away_pitcher = current_game['AwayStartingPitcher']
date = current_game['Date']
home_pitcher_last_name = home_pitcher.split()[-1]
away_pitcher_last_name = away_pitcher.split()[-1]
current_betting = betting_data.loc[(betting_data.homePitcher == home_pitcher_last_name) &
                                    (betting_data.awayPitcher == away_pitcher_last_name) &
                                    (betting_data.date == str(date))]
print(current_betting)

         date homeTeam awayTeam  homeScore  awayScore homePitcher awayPitcher  \
997  20170424      ARI       SD        7.0        6.0     Greinke      Chacin   

     homeLine  awayLine  
997    -144.0     133.0  


In [12]:
rows_list = []
for i in range(training_set.shape[0]):
    if (i%25==0):
        print (i)
    current_game = training_set.iloc[i]
    current_home = current_game['HomeTeam']
    current_away = current_game['AwayTeam']
    home_game_number = current_game['HomeTeamGameNumber']
    away_game_number = current_game['AwayTeamGameNumber']
    date = current_game['Date']
    home_pitcher = current_game['HomeStartingPitcher']
    away_pitcher = current_game['AwayStartingPitcher']
#     home_pitcher_last_name = home_pitcher.split()[-1]
#     away_pitcher_last_name = away_pitcher.split()[-1]
#     current_betting_data = betting_data.loc[(betting_data.homePitcher == home_pitcher_last_name) &
#                                         (betting_data.awayPitcher == away_pitcher_last_name) &
#                                         (betting_data.date == str(date))]
    
    #all previous games
    all_prev_home = get_last_n_games(current_home, home_game_number-1, home_game_number, training_set)
    all_prev_away = get_last_n_games(current_away, away_game_number-1, away_game_number, training_set)
    
    #last 20 games of home team
    last20home = get_last_n_games(current_home, 20, home_game_number, training_set)
    #last 20 games of away team
    last20away = get_last_n_games(current_away, 20, away_game_number, training_set)
    
    #last 10 games of home team
    last10home = get_last_n_games(current_home, 10, home_game_number, training_set)
    #last 10 games of away team
    last10away = get_last_n_games(current_away, 10, away_game_number, training_set)
    
    #last 5 games of home team
    last5home = get_last_n_games(current_home, 5, home_game_number, training_set)
    #last 5 games of away team
    last5away = get_last_n_games(current_away, 5, away_game_number, training_set)
    
    rows_list.append({
            'date' : date,
            'home_team': current_home,
            'away_team': current_away,
            'home_score' : current_game['HomeScore'],
            'away_score' : current_game['AwayScore'],
#             'home_line' : (current_betting_data['homeLine'].values)[0],
#             'away_line' : (current_betting_data['awayLine'].values)[0],
            
            #20 game hitting stats
            'home_20GameAvgBattingAvg': return_rolling_average_of_column(last20home, current_home, "BA"),
            'away_20GameAvgBattingAvg': return_rolling_average_of_column(last20away, current_away, "BA"),
            'home_20GameAvgRunsPerGame': return_rolling_average_of_column(last20home, current_home, "Runs"),
            'away_20GameAvgRunsPerGame': return_rolling_average_of_column(last20away, current_away, "Runs"),
            'home_20GameAvgWalksPerGame': return_rolling_average_of_column(last20home, current_home, "Walks"),
            'away_20GameAvgWalksPerGame': return_rolling_average_of_column(last20away, current_away, "Walks"),
            'home_20GameAvgStrikeoutsPerGame': return_rolling_average_of_column(last20home, current_home, "Strikeouts"),
            'away_20GameAvgStrikeoutsPerGame': return_rolling_average_of_column(last20away, current_away, "Strikeouts"),
            'home_20GameAvgSBPerGame': return_rolling_average_of_column(last20home, current_home, "StolenBases"),
            'away_20GameAvgSBPerGame': return_rolling_average_of_column(last20away, current_away, "StolenBases"),
            'home_20GameAvgLOBPerGame': return_rolling_average_of_column(last20home, current_home, "LOB"),
            'away_20GameAvgLOBPerGame': return_rolling_average_of_column(last20away, current_away, "LOB"),
            'home_20GameOPS' : return_rolling_average_of_column(last20home, current_home, "OPS"),
            'away_20GameOPS' : return_rolling_average_of_column(last20away, current_away, "OPS"),
            
            #10 game hitting stats
            'home_10GameAvgBattingAvg': return_rolling_average_of_column(last10home, current_home, "BA"),
            'away_10GameAvgBattingAvg': return_rolling_average_of_column(last10away, current_away, "BA"),
            'home_10GameAvgRunsPerGame': return_rolling_average_of_column(last10home, current_home, "Runs"),
            'away_10GameAvgRunsPerGame': return_rolling_average_of_column(last10away, current_away, "Runs"),
            'home_10GameAvgWalksPerGame': return_rolling_average_of_column(last10home, current_home, "Walks"),
            'away_10GameAvgWalksPerGame': return_rolling_average_of_column(last10away, current_away, "Walks"),
            'home_10GameAvgStrikeoutsPerGame': return_rolling_average_of_column(last10home, current_home, "Strikeouts"),
            'away_10GameAvgStrikeoutsPerGame': return_rolling_average_of_column(last10away, current_away, "Strikeouts"),
            'home_10GameAvgSBPerGame': return_rolling_average_of_column(last10home, current_home, "StolenBases"),
            'away_10GameAvgSBPerGame': return_rolling_average_of_column(last10away, current_away, "StolenBases"),
            'home_10GameAvgLOBPerGame': return_rolling_average_of_column(last10home, current_home, "LOB"),
            'away_10GameAvgLOBPerGame': return_rolling_average_of_column(last10away, current_away, "LOB"),
            'home_10GameOPS' : return_rolling_average_of_column(last10home, current_home, "OPS"),
            'away_10GameOPS' : return_rolling_average_of_column(last10away, current_away, "OPS"),
            
            #5 game hitting stats
            'home_5GameAvgBattingAvg': return_rolling_average_of_column(last5home, current_home, "BA"),
            'away_5GameAvgBattingAvg': return_rolling_average_of_column(last5away, current_away, "BA"),
            'home_5GameAvgRunsPerGame': return_rolling_average_of_column(last5home, current_home, "Runs"),
            'away_5GameAvgRunsPerGame': return_rolling_average_of_column(last5away, current_away, "Runs"),
            'home_5GameAvgWalksPerGame': return_rolling_average_of_column(last5home, current_home, "Walks"),
            'away_5GameAvgWalksPerGame': return_rolling_average_of_column(last5away, current_away, "Walks"),
            'home_5GameAvgStrikeoutsPerGame': return_rolling_average_of_column(last5home, current_home, "Strikeouts"),
            'away_5GameAvgStrikeoutsPerGame': return_rolling_average_of_column(last5away, current_away, "Strikeouts"),
            'home_5GameAvgSBPerGame': return_rolling_average_of_column(last5home, current_home, "StolenBases"),
            'away_5GameAvgSBPerGame': return_rolling_average_of_column(last5away, current_away, "StolenBases"),
            'home_5GameAvgLOBPerGame': return_rolling_average_of_column(last5home, current_home, "LOB"),
            'away_5GameAvgLOBPerGame': return_rolling_average_of_column(last5away, current_away, "LOB"),
            'home_5GameOPS' : return_rolling_average_of_column(last5home, current_home, "OPS"),
            'away_5GameOPS' : return_rolling_average_of_column(last5away, current_away, "OPS"),
        
            #pitching stats
            'home_pitcher' : home_pitcher,
            'away_pitcher' : away_pitcher,
            'home_pitcher_strikeouts' : return_cumulative_average_of_pitching_column(all_prev_home, current_home, 'Strikeouts', home_pitcher),
            'away_pitcher_strikeouts' : return_cumulative_average_of_pitching_column(all_prev_away, current_away, 'Strikeouts', away_pitcher),
            'home_pitcher_walks' : return_cumulative_average_of_pitching_column(all_prev_home, current_home, 'Walks', home_pitcher),
            'away_pitcher_walks' : return_cumulative_average_of_pitching_column(all_prev_away, current_away, 'Walks', away_pitcher),
            'home_pitcher_hits' : return_cumulative_average_of_pitching_column(all_prev_home, current_home, 'Hits', home_pitcher),
            'away_pitcher_hits' : return_cumulative_average_of_pitching_column(all_prev_away, current_away, 'Hits', away_pitcher),
            'home_pitcher_ERA' : return_cumulative_average_of_pitching_column(all_prev_home, current_home, 'ERA', home_pitcher),
            'away_pitcher_ERA' : return_cumulative_average_of_pitching_column(all_prev_away, current_away, 'ERA', away_pitcher)

        })
    
f = pd.DataFrame(rows_list)
f.to_csv('test.csv', 
         index=None, 
         columns = ['date','home_team', 'away_team', 'home_score', 'away_score', 
                    'home_20GameAvgBattingAvg', 'home_10GameAvgBattingAvg','home_5GameAvgBattingAvg',
                    'home_20GameAvgRunsPerGame','home_10GameAvgRunsPerGame','home_5GameAvgRunsPerGame',
                    'home_20GameAvgWalksPerGame','home_10GameAvgWalksPerGame','home_5GameAvgWalksPerGame',
                    'home_20GameAvgStrikeoutsPerGame','home_10GameAvgStrikeoutsPerGame','home_5GameAvgStrikeoutsPerGame',
                    'home_20GameAvgSBPerGame','home_10GameAvgSBPerGame', 'home_5GameAvgSBPerGame',
                    'home_20GameAvgLOBPerGame','home_10GameAvgLOBPerGame','home_5GameAvgLOBPerGame',
                    'home_20GameOPS','home_10GameOPS','home_5GameOPS',
                    'home_pitcher', 'home_pitcher_strikeouts', 'home_pitcher_walks', 'home_pitcher_hits', 'home_pitcher_ERA',
                    'away_20GameAvgBattingAvg','away_10GameAvgBattingAvg','away_5GameAvgBattingAvg',
                    'away_20GameAvgRunsPerGame','away_10GameAvgRunsPerGame','away_5GameAvgRunsPerGame',
                    'away_20GameAvgWalksPerGame','away_10GameAvgWalksPerGame', 'away_5GameAvgWalksPerGame',
                    'away_20GameAvgStrikeoutsPerGame','away_10GameAvgStrikeoutsPerGame', 'away_5GameAvgStrikeoutsPerGame',
                    'away_20GameAvgSBPerGame', 'away_10GameAvgSBPerGame', 'away_5GameAvgSBPerGame',
                    'away_20GameAvgLOBPerGame', 'away_10GameAvgLOBPerGame','away_5GameAvgLOBPerGame',
                    'away_20GameOPS', 'away_10GameOPS', 'away_5GameOPS', 
                    'away_pitcher', 'away_pitcher_strikeouts', 'away_pitcher_walks', 'away_pitcher_hits', 'away_pitcher_ERA' 
                   ])

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100


In [13]:
f = pd.DataFrame(rows_list)

In [14]:
print(f.shape)
print(betting_data.shape)

(2123, 57)
(2468, 9)


In [15]:
training_set.head()

,Date,Day,AwayTeam,AwayTeamGameNumber,HomeTeam,HomeTeamGameNumber,AwayScore,HomeScore,AwayAtBats,AwayHits,...,HomeWalks,HomeIntentionalWalks,HomeStrikeouts,HomeStolenBases,HomeCaughtStealing,HomeLeftOnBase,HomeIndividualERA,HomeTeamERA,HomeErrors,HomeStartingPitcher
280,20170424,Mon,SDN,21,ARI,21,6,7,37,10,...,2,0,8,4,0,6,5,5,1,Zack Greinke
289,20170425,Tue,SDN,22,ARI,22,3,9,37,10,...,3,0,8,2,1,9,3,3,0,Patrick Corbin
291,20170425,Tue,CIN,21,MIL,22,1,9,35,8,...,7,1,6,3,1,8,1,1,1,Zach Davies
293,20170425,Tue,LAN,21,SFN,21,2,1,32,7,...,1,0,9,1,0,8,2,2,0,Ty Blach
301,20170426,Wed,SDN,23,ARI,23,8,5,37,12,...,5,0,13,3,1,10,8,8,1,Zack Godley
